<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
import pprint
import torch

torch.cuda.is_available()

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
!pip install pytorch-pretrained-bert
  
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']
  
!test -d data || mkdir data mkdir data/twitter; mkdir data/twitter/tmp
 


Cloning into 'bert_repo'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 234.47 KiB | 845.00 KiB/s, done.
Resolving deltas: 100% (186/186), done.
     |████████████████████████████████| 133kB 9.2MB/s 


# This takes a while dont run if you wont use fp16

In [0]:
! test -d ./apex || git clone https://github.com/NVIDIA/apex 
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

if not 'apex' in sys.path:
  sys.path += ['apex']

### Prepare and import BERT and other modules


In [0]:
from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
from pathlib import Path
import torch
import re
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
from fastai.text import Tokenizer, Vocab
import pandas as pd
import collections
import os
import pdb
from tqdm import tqdm, trange
import sys
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
import apex
from sklearn.model_selection import train_test_split
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.metrics import roc_curve, auc


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
class BertForMultiLabelClassification(BertPreTrainedModel):
  """BERT model for classification.
  This module is composed of the BERT model with a linear layer on top of
  the pooled output.
  Params:
      `config`: a BertConfig class instance with the configuration to build a new model.
      `num_labels`: the number of classes for the classifier. Default = 2.
  Inputs:
      `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
          with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
          `extract_features.py`, `run_classifier.py` and `run_squad.py`)
      `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
          types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
          a `sentence B` token (see BERT paper for more details).
      `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
          selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
          input sequence length in the current batch. It's the mask that we typically use for attention when
          a batch has varying length sentences.
      `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
          with indices selected in [0, ..., num_labels].
  Outputs:
      if `labels` is not `None`:
          Outputs the CrossEntropy classification loss of the output with the labels.
      if `labels` is `None`:
          Outputs the classification logits of shape [batch_size, num_labels].
  Example usage:
  ```python
  # Already been converted into WordPiece token ids
  input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
  input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
  token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
  config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
      num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
  num_labels = 2
  model = BertForSequenceClassification(config, num_labels)
  logits = model(input_ids, token_type_ids, input_mask)
  ```
  """
  def __init__(self, config, num_labels=4):
    super(BertForMultiLabelClassification, self).__init__(config)
    self.num_labels = num_labels
    self.bert = BertModel(config)
    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
    self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
    self.apply(self.init_bert_weights)

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
    _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)

    if labels is not None:
      loss_fct = BCEWithLogitsLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
      return loss
    else:
      return logits

  def freeze_bert_encoder(self):
    for param in self.bert.parameters():
      param.requires_grad = False

  def unfreeze_bert_encoder(self):
    for param in self.bert.parameters():
      param.requires_grad = True

In [0]:
"""The following code overwrites some core BERT functionality"""

class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
      sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
      Only must be specified for sequence pair tasks.
      labels: (Optional) [string]. The label of the example. This should be
      specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, input_ids, input_mask, segment_ids, label_id):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id

class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir, data_file_name, size=-1):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError() 

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()
        
class MultilabelProcessor(DataProcessor):
  """Custom Processor for the Myers-Briggs Data.

   Assumes input data as follows:

    id at index 0,text at index 1, labels at [2:]

   Example for our problem:
      0,sometext,0,1,1,1
      1,moretext,1,0,0,1
    OR
      ENFJ,sometext,0,1,1,1
      ENFJ,moretext,0,1,1,1

   Meaning - id does not need to be unique, it can be used as a label
   to try multi-class solution of the same task.
  """
  
  
  def __init__(self, labels=None):
    """Constructs a MultilabelProcessor
    
    Args:
      labels: array-like or list. Columns under which labels are stored in 
      the input dataframes. Default is None: assumes labels are [2:]
    """
    
    self.labels = labels

  def get_train_examples(self, data_dir):
    """Pass in train data by specifying it's directory. Looks for train.csv"""
    path = Path(data_dir, "train.csv")
    logger.info("LOOKING AT {}".format(path))
    return self._create_examples(pd.read_csv(path), "train")

  def get_dev_examples(self, data_dir):
    """Pass in val (development) data by specifying it's directory. Looks for val.csv."""
    path = Path(data_dir, "val.csv")
    logger.info("LOOKING AT {}".format(path))
    return self._create_examples(pd.read_csv(path), "val")

  def get_test_examples(self, data_dir, data_file_name, size=-1):
    """See base class."""
    data_df = pd.read_csv(os.path.join(data_dir, data_file_name))

    if size == -1:
      return self._create_examples(data_df, "test")
    else:
      return self._create_examples(data_df.sample(size), "val")

  def get_labels(self):
    """4 binary labels: IE, NS, FT, JP. 1 means I,N,F,J; 0 means E,S,T,P"""
    return self.labels

  def _create_examples(self, df, set_type, labels_available=True):
    """Creates examples for the train, val and test sets."""
    if len(df.columns) <= 2:
      raise IndexError("Invalid dataframe format. Must have guid, text, labels")
    if self.labels is None:
      self.labels = df.columns[2:]
      
    examples = []

    for i, row in enumerate(df.values):
      guid = "%s-%s-%s" % (i, set_type, row[0])
      text_a = row[1]
      if labels_available:
        labels = row[2:]
      else:
        labels = []
      examples.append(
        InputExample(guid=guid, text_a=text_a, label=labels))
    return examples
  
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode="classification"):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      logger.info("Writing example %d of %d" % (ex_index, len(examples)))

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambigiously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)

    if tokens_b:
      tokens += tokens_b + ["[SEP]"]
      segment_ids += [1] * (len(tokens_b) + 1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids += padding

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.label:
      labels_ids.append(int(label))

    if ex_index < 1:
      logger.info("*** Example ***")
      logger.info("guid: %s" % (example.guid))
      logger.info("tokens: %s" % " ".join(
              [str(x) for x in tokens]))
      logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      logger.info(
              "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
      logger.info("label: %s (id = %s)" % (example.label, labels_ids))

    features.append(InputFeatures(input_ids=input_ids,
                                  input_mask=input_mask,
                                  segment_ids=segment_ids,
                                  label_id=labels_ids,
                                 ))
  return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def accuracy(y_pred:Tensor, y_true:Tensor):
  """Simple accuracy. A bad metric for multilabel."""
  return (y_pred == y_true).sum() / len(y_true)

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
  """Compute accuracy when `y_pred` and `y_true` are the same size."""
  if sigmoid: 
    y_pred = y_pred.sigmoid()
  return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()
  
def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, 
                    eps:float=1e-9, sigmoid:bool=True):
  """Computes the f_beta between `preds` and `targets`."
    F1-metric version, `beta` controls importantce of precision vs recall,
    where beta=1 is equivalent to F1.
  """
  beta2 = beta ** 2
  if sigmoid: y_pred = y_pred.sigmoid()
  y_pred = (y_pred>thresh).float()
  y_true = y_true.float()
  TP = (y_pred*y_true).sum(dim=1)
  prec = TP/(y_pred.sum(dim=1)+eps)
  rec = TP/(y_true.sum(dim=1)+eps)
  res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
  return res.mean().item()



In [0]:
! cp test.csv data/twitter/tmp ; cp train.csv data/twitter/tmp ; cp val.csv data/twitter/tmp

In [0]:
"""Google's tensorflow training parameters"""


"""
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500


# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config


def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=ITERATIONS_PER_LOOP,
      num_shards=NUM_TPU_CORES,
      per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))
"""

"""Set pytorch parameters"""

LABEL_COLS=['I','N','F','J']

DATA=Path('./data')
DATA.mkdir(exist_ok=True)

DATA_PATH=DATA/'twitter'
DATA_PATH.mkdir(exist_ok=True)
BERT_MODEL = 'uncased_L-12_H-768_A-12'
PATH=DATA_PATH/'tmp'
PATH.mkdir(exist_ok=True)

CLAS_DATA_PATH=PATH/'class'
CLAS_DATA_PATH.mkdir(exist_ok=True)

model_state_dict = None

args = {
  "train_size": -1,
  "val_size": -1,
  "full_data_dir": DATA_PATH,
  "data_dir": PATH,
  "task_name": "twitter",
  "labels": LABEL_COLS,
  "no_cuda": False,
  "bert_model": 'bert-base-uncased',
  "output_dir": CLAS_DATA_PATH/'output',
  "max_seq_length": 128,
  "do_train": True,
  "do_eval": True,
  "do_lower_case": True,
  "train_batch_size": 16,
  "eval_batch_size": 8,
  "learning_rate": 3e-5,
  "num_train_epochs": 4.0,
  "warmup_proportion": 0.1,
  "local_rank": -1,
  "seed": 42,
  "gradient_accumulation_steps": 1,
  "optimize_on_cpu": False,
  "fp16": False,
  "loss_scale": 128,
  
}


#
if args["local_rank"] == -1 or args["no_cuda"]:
  device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")
  n_gpu = torch.cuda.device_count()
#     n_gpu = 1
else:
  torch.cuda.set_device(args['local_rank'])
  device = torch.device("cuda", args['local_rank'])
  n_gpu = 1
  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
  torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args['local_rank'] != -1), args['fp16']))

processors = {
  "twitter": MultilabelProcessor
}

args['train_batch_size'] = int(args['train_batch_size'] / args['gradient_accumulation_steps'])

random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
task_name = args['task_name'].lower()

if task_name not in processors:
  raise ValueError("Task not found: %s" % (task_name))

label_list=args['labels']
processor = processors[task_name](label_list)
num_labels = len(args['labels'])



04/29/2019 09:51:12 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [0]:
tokenizer = BertTokenizer.from_pretrained(args['bert_model'], do_lower_case=args['do_lower_case'])

logger.info("***** Loading data *****")

train_examples = None
num_train_steps = None
if args['do_train']:
  train_examples = processor.get_train_examples(args['full_data_dir'])
  num_train_steps = int(
      len(train_examples)/args['train_batch_size']/args['gradient_accumulation_steps']*args['num_train_epochs']
  )
eval_examples = processor.get_dev_examples(args['data_dir'])
train_features = convert_examples_to_features(train_examples, args['labels'], args['max_seq_length'], tokenizer)


logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args['train_batch_size'])
logger.info("  Num steps = %d", num_train_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
if args['local_rank'] == -1:
  train_sampler = RandomSampler(train_data)
else:
  train_sampler = DistributedSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args['train_batch_size'])

04/29/2019 09:51:12 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/29/2019 09:51:12 - INFO - __main__ -   ***** Loading data *****
04/29/2019 09:51:12 - INFO - __main__ -   LOOKING AT data/twitter/train.csv
04/29/2019 09:51:12 - INFO - __main__ -   LOOKING AT data/twitter/tmp/val.csv
04/29/2019 09:51:12 - INFO - __main__ -   Writing example 0 of 2963
04/29/2019 09:51:12 - INFO - __main__ -   *** Example ***
04/29/2019 09:51:12 - INFO - __main__ -   guid: 0-train-0
04/29/2019 09:51:12 - INFO - __main__ -   tokens: [CLS] i ' m attending @ user guide to a better blog ebook launch party for a chance to win $ 300 gift card ! @ hash ##tag @ ur ##l / 9 ##n ##15 ##b ##v ##lt ##ep rs ##v ##p to the @ user guide to 

In [0]:
def get_model():
  model = BertForMultiLabelClassification.from_pretrained(args['bert_model'], num_labels = num_labels)
  return model

model = get_model()



if args['fp16']:
  model.half()
model.to(device)

04/29/2019 09:51:24 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/29/2019 09:51:24 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpqjmsyyq8
04/29/2019 09:51:28 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size

BertForMultiLabelClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, ele

In [0]:
from torch.optim.lr_scheduler import _LRScheduler, Optimizer

def warmup_linear(x, warmup=0.002):
  """Warmup for how long?"""
  if x < warmup:
      return x/warmup
  return 1.0 - x

class CyclicLR(object):
  """Sets the learning rate of each parameter group according to
  cyclical learning rate policy (CLR). The policy cycles the learning
  rate between two boundaries with a constant frequency, as detailed in
  the paper `Cyclical Learning Rates for Training Neural Networks`_.
  The distance between the two boundaries can be scaled on a per-iteration
  or per-cycle basis.
  Cyclical learning rate policy changes the learning rate after every batch.
  `batch_step` should be called after a batch has been used for training.
  To resume training, save `last_batch_iteration` and use it to instantiate `CycleLR`.
  This class has three built-in policies, as put forth in the paper:
  "triangular":
      A basic triangular cycle w/ no amplitude scaling.
  "triangular2":
      A basic triangular cycle that scales initial amplitude by half each cycle.
  "exp_range":
      A cycle that scales initial amplitude by gamma**(cycle iterations) at each
      cycle iteration.
  This implementation was adapted from the github repo: `bckenstler/CLR`_
  Args:
      optimizer (Optimizer): Wrapped optimizer.
      base_lr (float or list): Initial learning rate which is the
          lower boundary in the cycle for eachparam groups.
          Default: 0.001
      max_lr (float or list): Upper boundaries in the cycle for
          each parameter group. Functionally,
          it defines the cycle amplitude (max_lr - base_lr).
          The lr at any cycle is the sum of base_lr
          and some scaling of the amplitude; therefore
          max_lr may not actually be reached depending on
          scaling function. Default: 0.006
      step_size (int): Number of training iterations per
          half cycle. Authors suggest setting step_size
          2-8 x training iterations in epoch. Default: 2000
      mode (str): One of {triangular, triangular2, exp_range}.
          Values correspond to policies detailed above.
          If scale_fn is not None, this argument is ignored.
          Default: 'triangular'
      gamma (float): Constant in 'exp_range' scaling function:
          gamma**(cycle iterations)
          Default: 1.0
      scale_fn (function): Custom scaling policy defined by a single
          argument lambda function, where
          0 <= scale_fn(x) <= 1 for all x >= 0.
          mode paramater is ignored
          Default: None
      scale_mode (str): {'cycle', 'iterations'}.
          Defines whether scale_fn is evaluated on
          cycle number or cycle iterations (training
          iterations since start of cycle).
          Default: 'cycle'
      last_batch_iteration (int): The index of the last batch. Default: -1
  Example:
      >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
      >>> scheduler = torch.optim.CyclicLR(optimizer)
      >>> data_loader = torch.utils.data.DataLoader(...)
      >>> for epoch in range(10):
      >>>     for batch in data_loader:
      >>>         scheduler.batch_step()
      >>>         train_batch(...)
  .. _Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
  .. _bckenstler/CLR: https://github.com/bckenstler/CLR
  """

  def __init__(self,
               optimizer,
               base_lr=1e-3,
               max_lr=6e-3,
               step_size=2000,
               mode='triangular',
               gamma=1.,
               scale_fn=None,
               scale_mode='cycle',
               last_batch_iteration=-1,
              ):
    
    self.optimizer = optimizer

    if isinstance(base_lr, list) or isinstance(base_lr, tuple):
      if len(base_lr) != len(optimizer.param_groups):
        raise ValueError("expected {} base_lr, got {}".format(
          len(optimizer.param_groups), len(base_lr)))
      self.base_lrs = list(base_lr)
    else:
      self.base_lrs = [base_lr] * len(optimizer.param_groups)

    if isinstance(max_lr, list) or isinstance(max_lr, tuple):
      if len(max_lr) != len(optimizer.param_groups):
        raise ValueError("expected {} max_lr, got {}".format(
          len(optimizer.param_groups), len(max_lr)))
      self.max_lrs = list(max_lr)
    else:
      self.max_lrs = [max_lr] * len(optimizer.param_groups)

    self.step_size = step_size

    if mode not in ['triangular', 'triangular2', 'exp_range'] and scale_fn is None:
       raise ValueError('mode is invalid and scale_fn is None')

    self.mode = mode
    self.gamma = gamma

    if scale_fn is None:
      if self.mode == 'triangular':
        self.scale_fn = self._triangular_scale_fn
        self.scale_mode = 'cycle'
      elif self.mode == 'triangular2':
        self.scale_fn = self._triangular2_scale_fn
        self.scale_mode = 'cycle'
      elif self.mode == 'exp_range':
        self.scale_fn = self._exp_range_scale_fn
        self.scale_mode = 'iterations'
    else:
      self.scale_fn = scale_fn
      self.scale_mode = scale_mode

    self.batch_step(last_batch_iteration + 1)
    self.last_batch_iteration = last_batch_iteration

  def batch_step(self, batch_iteration=None):
    if batch_iteration is None:
      batch_iteration = self.last_batch_iteration + 1
    self.last_batch_iteration = batch_iteration
    for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
      param_group['lr'] = lr

  def _triangular_scale_fn(self, x):
    return 1.

  def _triangular2_scale_fn(self, x):
    return 1 / (2. ** (x - 1))

  def _exp_range_scale_fn(self, x):
    return self.gamma**(x)

  def get_lr(self):
    step_size = float(self.step_size)
    cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
    x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

    lrs = []
    param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
    for param_group, base_lr, max_lr in param_lrs:
      base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
      if self.scale_mode == 'cycle':
        lr = base_lr + base_height * self.scale_fn(cycle)
      else:
        lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
      lrs.append(lr)
    return lrs
  
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
  ]

t_total = num_train_steps

if args['local_rank'] != -1:
  t_total = t_total // torch.distributed.get_world_size()

bert_adam=BertAdam(optimizer_grouped_parameters,lr=args['learning_rate'],warmup=args['warmup_proportion'],t_total=t_total)

scheduler = CyclicLR(optimizer=bert_adam, base_lr=2e-5, max_lr=5e-5, step_size=2500, last_batch_iteration=0)

In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps
if args['local_rank'] != -1:
    t_total = t_total // torch.distributed.get_world_size()
if args['fp16']:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=args['learning_rate'],
                          bias_correction=False,
                          max_grad_norm=1.0)
    if args['loss_scale'] == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=args['loss_scale'])

else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=args['learning_rate'],
                         warmup=args['warmup_proportion'],
                         t_total=t_total)

scheduler = CyclicLR(optimizer, base_lr=2e-5, max_lr=5e-5, step_size=2500, last_batch_iteration=0)

## Validation function

In [0]:

def eval():
  args['output_dir'].mkdir(exist_ok=True)

  eval_features = convert_examples_to_features(eval_examples, args['labels'], args['max_seq_length'], tokenizer)
  logger.info("***** Running evaluation *****")
  logger.info("  Num examples = %d", len(eval_examples))
  logger.info("  Batch size = %d", args['eval_batch_size'])
  all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
  all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
  all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
  all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)
  eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  # Run prediction for full data
  eval_sampler = SequentialSampler(eval_data)
  eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args['eval_batch_size'])

  all_logits = None
  all_labels = None

  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  for input_ids, input_mask, segment_ids, label_id in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_id = label_id.to(device)

    with torch.no_grad():
      tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_id)
      logits = model(input_ids, segment_ids, input_mask)


    tmp_eval_accuracy = accuracy_thresh(logits, label_id)
    if all_logits is None:
      all_logits = logits.detach().cpu().numpy()
    else:
      all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)

    if all_labels is None:
      all_labels = label_id.detach().cpu().numpy()
    else:    
      all_labels = np.concatenate((all_labels, label_id.detach().cpu().numpy()), axis=0)


  eval_loss += tmp_eval_loss.mean().item()
  eval_accuracy += tmp_eval_accuracy

  nb_eval_examples += input_ids.size(0)
  nb_eval_steps += 1

  eval_loss = eval_loss / nb_eval_steps
  eval_accuracy = eval_accuracy / nb_eval_examples

  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()

  for i in range(num_labels):
    fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], all_logits[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(all_labels.ravel(), all_logits.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  result = {'eval_loss': eval_loss,
            'eval_accuracy': eval_accuracy,
            'roc_auc': roc_auc  }

  output_eval_file = os.path.join(args['output_dir'], "eval_results.txt")
  with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in sorted(result.keys()):
      logger.info("  %s = %s", key, str(result[key]))
  return result

## Train model

In [0]:

def fit(num_epocs=args['num_train_epochs']):
  global_step = 0
  model.train()
  for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

      batch = tuple(t.to(device) for t in batch)
      input_ids, input_mask, segment_ids, label_id = batch
      loss = model(input_ids, segment_ids, input_mask, label_id)
      if n_gpu > 1:
        loss = loss.mean() # mean() to average on multi-gpu.
      if args['gradient_accumulation_steps'] > 1:
        loss = loss / args['gradient_accumulation_steps']

      if args['fp16']:
        optimizer.backward(loss)
      else:
        loss.backward()

      tr_loss += loss.item()
      nb_tr_examples += input_ids.size(0)
      nb_tr_steps += 1
      if (step + 1) % args['gradient_accumulation_steps'] == 0:
        # modify learning rate with special warm up BERT uses
        lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
        for param_group in optimizer.param_groups:
          param_group['lr'] = lr_this_step
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

    logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
    logger.info('Eval after epoc {}'.format(i_+1))
    eval()

In [0]:
model.unfreeze_bert_encoder()

In [0]:
fit()

04/29/2019 09:53:03 - INFO - __main__ -   Loss after epoc 0.6897769262713771
04/29/2019 09:53:03 - INFO - __main__ -   Eval after epoc 1
04/29/2019 09:53:03 - INFO - __main__ -   Writing example 0 of 329
04/29/2019 09:53:03 - INFO - __main__ -   *** Example ***
04/29/2019 09:53:03 - INFO - __main__ -   guid: 0-val-1823
04/29/2019 09:53:03 - INFO - __main__ -   tokens: [CLS] at it . @ user what scenery / because its a desert ha ##ha ##ha ##ha / get it ? / cu ##z its just dirt and rocks ha ##ha . . ah . . ha @ hash ##tag spending my morning waiting to get screwed by com ##cast @ user id ##k ##lo ##l i feel like ask ##in someone to do free any ##thi ##g for ya is a lil rude . " hey man can you fix my transmission for free lo ##l ty " @ user but in the same breath its like , why we ma ##kin a ps ##a outta " free art = bad and you are terrible so stop " is kinda dumb and funny . @ user [SEP]
04/29/2019 09:53:03 - INFO - __main__ -   input_ids: 101 2012 2009 1012 1030 5310 2054 17363 1013 21

04/29/2019 09:54:35 - INFO - __main__ -   Loss after epoc 0.6547121242169411
04/29/2019 09:54:35 - INFO - __main__ -   Eval after epoc 2
04/29/2019 09:54:35 - INFO - __main__ -   Writing example 0 of 329
04/29/2019 09:54:35 - INFO - __main__ -   *** Example ***
04/29/2019 09:54:35 - INFO - __main__ -   guid: 0-val-1823
04/29/2019 09:54:35 - INFO - __main__ -   tokens: [CLS] at it . @ user what scenery / because its a desert ha ##ha ##ha ##ha / get it ? / cu ##z its just dirt and rocks ha ##ha . . ah . . ha @ hash ##tag spending my morning waiting to get screwed by com ##cast @ user id ##k ##lo ##l i feel like ask ##in someone to do free any ##thi ##g for ya is a lil rude . " hey man can you fix my transmission for free lo ##l ty " @ user but in the same breath its like , why we ma ##kin a ps ##a outta " free art = bad and you are terrible so stop " is kinda dumb and funny . @ user [SEP]
04/29/2019 09:54:35 - INFO - __main__ -   input_ids: 101 2012 2009 1012 1030 5310 2054 17363 1013 21

04/29/2019 09:56:08 - INFO - __main__ -   Loss after epoc 0.5166947707053153
04/29/2019 09:56:08 - INFO - __main__ -   Eval after epoc 3
04/29/2019 09:56:08 - INFO - __main__ -   Writing example 0 of 329
04/29/2019 09:56:08 - INFO - __main__ -   *** Example ***
04/29/2019 09:56:08 - INFO - __main__ -   guid: 0-val-1823
04/29/2019 09:56:08 - INFO - __main__ -   tokens: [CLS] at it . @ user what scenery / because its a desert ha ##ha ##ha ##ha / get it ? / cu ##z its just dirt and rocks ha ##ha . . ah . . ha @ hash ##tag spending my morning waiting to get screwed by com ##cast @ user id ##k ##lo ##l i feel like ask ##in someone to do free any ##thi ##g for ya is a lil rude . " hey man can you fix my transmission for free lo ##l ty " @ user but in the same breath its like , why we ma ##kin a ps ##a outta " free art = bad and you are terrible so stop " is kinda dumb and funny . @ user [SEP]
04/29/2019 09:56:08 - INFO - __main__ -   input_ids: 101 2012 2009 1012 1030 5310 2054 17363 1013 21

04/29/2019 09:57:40 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
04/29/2019 09:57:40 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
04/29/2019 09:57:41 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
04/29/2019 09:57:41 - INFO - __main__ -   Loss after epoc 0.3919578013561105
04/29/2019 09:57:41 - INFO - __main__ -   Eval after epoc 4
04/29/2019 09:57:41 - INFO - __main__ -   Writing example 0 of 329
04/29/2019 09:57:41 - INFO - __main__ -   *** Example ***
04/29/2019 09:57:41 - INFO - __main__ -   guid: 0-val-1823
04/29/2019 09:57:41 - INFO - __main__ -   tokens: [CLS] at it . @ user

## Save pretrained model 

In [0]:
! mkdir data/twitter/models

PYTORCH_PRETRAINED_BERT_CACHE=Path('./data/twitter/models')

In [0]:
# Save a trained model
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(PYTORCH_PRETRAINED_BERT_CACHE, "finetuned_pytorch_model.bin")
torch.save(model_to_save.state_dict(), output_model_file)



## Reload it

In [0]:
# Load a trained model that you have fine-tuned
model_state_dict = torch.load(output_model_file)
model = BertForMultiLabelClassification.from_pretrained(args['bert_model'], num_labels = num_labels, state_dict=model_state_dict)
model.to(device)

## Run on val

In [0]:
eval()

04/29/2019 10:25:15 - INFO - __main__ -   Writing example 0 of 329
04/29/2019 10:25:15 - INFO - __main__ -   *** Example ***
04/29/2019 10:25:15 - INFO - __main__ -   guid: 0-val-1823
04/29/2019 10:25:15 - INFO - __main__ -   tokens: [CLS] at it . @ user what scenery / because its a desert ha ##ha ##ha ##ha / get it ? / cu ##z its just dirt and rocks ha ##ha . . ah . . ha @ hash ##tag spending my morning waiting to get screwed by com ##cast @ user id ##k ##lo ##l i feel like ask ##in someone to do free any ##thi ##g for ya is a lil rude . " hey man can you fix my transmission for free lo ##l ty " @ user but in the same breath its like , why we ma ##kin a ps ##a outta " free art = bad and you are terrible so stop " is kinda dumb and funny . @ user [SEP]
04/29/2019 10:25:15 - INFO - __main__ -   input_ids: 101 2012 2009 1012 1030 5310 2054 17363 1013 2138 2049 1037 5532 5292 3270 3270 3270 1013 2131 2009 1029 1013 12731 2480 2049 2074 6900 1998 5749 5292 3270 1012 1012 6289 1012 1012 529

{'eval_accuracy': tensor(0, device='cuda:0'),
 'eval_accuracy_thresh': 0.75,
 'eval_loss': 0.5105093121528625,
 'roc_auc': {0: 0.658961038961039,
  1: 0.6796400533254334,
  2: 0.6489574090505768,
  3: 0.6777661639540092,
  'micro': 0.6636812468925967}}

## Run on test

In [0]:
def predict(model, path, test_filename='test.csv'):
  predict_processor = MultilabelProcessor(path)
  test_examples = predict_processor.get_test_examples(path, test_filename, size=-1)

  # Hold input data for returning it 
  input_data = [{ 'id': input_example.guid, 'comment_text': input_example.text_a } for input_example in test_examples]

  test_features = convert_examples_to_features(test_examples, label_list, args['max_seq_length'], tokenizer)

  logger.info("***** Running prediction *****")
  logger.info("  Num examples = %d", len(test_examples))
  logger.info("  Batch size = %d", args['eval_batch_size'])

  all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
  all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
  all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

  test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)

  # Run prediction for full data
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args['eval_batch_size'])

  all_logits = None

  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
    input_ids, input_mask, segment_ids = batch
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)

    with torch.no_grad():
      logits = model(input_ids, segment_ids, input_mask)
      logits = logits.sigmoid()

    if all_logits is None:
      all_logits = logits.detach().cpu().numpy()
    else:
      all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

  return pd.merge(pd.DataFrame(input_data), pd.DataFrame(all_logits, columns=label_list), left_index=True, right_index=True)

In [0]:
result = predict(model, DATA_PATH)

04/29/2019 10:27:09 - INFO - __main__ -   Writing example 0 of 296
04/29/2019 10:27:09 - INFO - __main__ -   *** Example ***
04/29/2019 10:27:09 - INFO - __main__ -   guid: 0-test-0
04/29/2019 10:27:09 - INFO - __main__ -   tokens: [CLS] @ user @ user @ user would be great or @ user or law students for rep ##ro justice could be great fits in an airport and deeply sad i ' m not able to be standing with protesters in nyc . sending every io ##ta of @ ur ##l / p ##bt ##d ##w ##bb 9 ##ez justice department to open civil rights investigation in eric garner case rt usa @ ur ##l / rs ##0 ##h ##vn ##nh ##e ##q @ hash ##tag @ hash ##tag @ user mine too , it was so beneath human dignity how they treated him . his family deserves justice . we all do . members of congress do hands up ! gesture in pro ##ts ##et on [SEP]
04/29/2019 10:27:09 - INFO - __main__ -   input_ids: 101 1030 5310 1030 5310 1030 5310 2052 2022 2307 2030 1030 5310 2030 2375 2493 2005 16360 3217 3425 2071 2022 2307 16142 1999 201

In [0]:
result.shape

(296, 6)

In [0]:
result[LABEL_COLS].to_csv(DATA_PATH/'classification_result.csv', index=None)

In [0]:
result[LABEL_COLS]

,I,N,F,J
0,0.221900,0.738083,0.281688,0.172758
1,0.162330,0.410585,0.735914,0.898665
2,0.693617,0.456062,0.570172,0.779696
3,0.335347,0.526126,0.250279,0.333264
4,0.847586,0.493204,0.298861,0.151132
5,0.597096,0.479653,0.744491,0.740229
6,0.918423,0.607985,0.590625,0.202568
7,0.711891,0.784681,0.689125,0.845608
8,0.941358,0.458784,0.648079,0.649091
9,0.388873,0.866237,0.760528,0.733217
